In [34]:
# Puzzle 1

go_to = {
    "N|" : "S",
    "NL" : "E",
    "NJ" : "W",
    "S|" : "N",
    "SF" : "E", 
    "S7" : "W", 
    "WJ" : "N", 
    "W-" : "E",
    "W7" : "S",
    "EL" : "N",
    "E-" : "W", 
    "EF" : "S"
}

def get_data(filename):
    with open(filename, "r") as file:
        maps = []
        for line in file:
            line = line.strip()
            row = []
            for char in line:
                row.append(char)
            maps.append(row)
    return maps

def print_map(maps):
    for row in maps:
        for col in row:
            print(col, end = "")
        print()

def find_start(maps):
    for r in range(len(maps)):
        for c in range(len(maps[r])):
            if maps[r][c] == "S":
                return r, c

def is_out_of_map(maps, pos):
    r, c = pos
    if r < 0 or r >= len(maps) or c < 0 or c >= len(maps[0]):
        return True
    return False

def is_deadend(maps, pos, come_from):
    r, c = pos
    pipe = maps[r][c]
    if come_from == "S" and pipe not in ["|", "7", "F"]:
        return True
    elif come_from == "N" and pipe not in ["|", "L", "J"]:
        return True
    elif come_from == "W" and pipe not in ["-", "J", "7"]:
        return True
    elif come_from == "E" and pipe not in ["-", "L", "F"]:
        return True
    return False   
            
def find_next(maps, pos, come_from):
    global go_to
    if is_out_of_map(maps, pos):
        #print("Out of Map at", pos)
        return (-1, -1), "X"
    
    if is_deadend(maps, pos, come_from):
        #print("Dead End at", pos)
        return (-1, -1), "X"
    
    # Find next position depending on last and pipe
    r, c = pos
    pipe = maps[r][c]
    
    new_dir = go_to[come_from + pipe]
    if new_dir == "N":
        return (r - 1, c), "S"
    elif new_dir == "E":
        return (r, c + 1), "W"
    elif new_dir == "W":
        return (r, c - 1), "E"
    elif new_dir == "S":
        return (r + 1, c), "N"
    else:
        print("Error, Position, Pipe, last field do not match", pos, come_from, pipe)
    
def find_path(maps, pos, come_from):
    global starting_point
    path = [pos]
    while True:
        new_pos, new_come_from = find_next(maps, pos, come_from)
        if new_pos == (-1, -1):
            print("Error, path not found")
            return path, "Error"
        elif new_pos == starting_point:
            print("Found Starting Point")
            path.append(new_pos)
            return path, "OK"
        else:
            pos = new_pos
            come_from = new_come_from
            path.append(pos)



# Main Program Puzzle 1
maps = get_data("Puzzle.txt")
starting_point = find_start(maps)

#print_map(maps)

#print(starting_point)
    
r, c = starting_point
second_points = [((r, c + 1), "W"), ((r, c - 1), "E"), ((r - 1, c), "S"), ((r + 1, c), "N")]
for sp in second_points:
    path, Error_Code = find_path(maps, sp[0], sp[1])
    if Error_Code == "OK":
        #print(path)
        print("Max Distance:", len(path)/2)
        second_pos = sp[0]
        second_come_from = sp[1]
        #print(second_pos, second_come_from)
        break

# Puzzle 2 starts here   
def clean_maps(maps, path):
    for r in range(len(maps)):
        for c in range(len(maps[0])):
            if (r, c) not in path:
                maps[r][c] = "."
    return maps

def mark_right_left_fields(maps, path, pos, come_from):
    while True:
        print(pos)
        r, c = pos
        if come_from == "S":
            if not(is_out_of_map(maps, (r,c + 1))) and not((r, c + 1) in path):
                maps[r][c + 1] = "r"
            if not(is_out_of_map(maps, (r, c - 1))) and not((r, c - 1) in path):
                maps[r][c + 1] = "l"
        elif come_from == "N":
            if not(is_out_of_map(maps, (r,c + 1))) and not((r, c + 1) in path):
                maps[r][c + 1] = "l"
            if not(is_out_of_map(maps, (r, c - 1))) and not((r, c - 1) in path):
                maps[r][c + 1] = "r"
        elif come_from == "W":
            if not(is_out_of_map(maps, (r + 1, c))) and not((r + 1, c) in path):
                maps[r + 1][c] = "l"
            if not(is_out_of_map(maps, (r - 1, c))) and not((r - 1, c) in path):
                maps[r - 1][c] = "r"
        elif come_from == "E":
            if not(is_out_of_map(maps, (r + 1, c))) and not((r + 1, c) in path):
                maps[r + 1][c] = "r"
            if not(is_out_of_map(maps, (r - 1, c))) and not((r - 1, c) in path):
                maps[r - 1][c] = "l"
            
        new_pos, new_come_from = find_next(maps, pos, come_from)

        if new_pos == starting_point:
            print("Found Starting Point")
            return maps
        else:
            pos = new_pos
            come_from = new_come_from

def is_inside(maps, path, pos):
    # Ray Tracing
    row, col = pos
    crossings = 0
    came_from = ""
    for new_col in range(col, len(maps[0])):
        if maps[row][new_col] == "|":
            crossings += 1
        elif maps[row][new_col] == "F":
            came_from = "S"
        elif maps[row][new_col] == "L":
            came_from = "N"
        elif maps[row][new_col] == "7" and came_from == "N":
            crossings += 1
        elif maps[row][new_col] == "J" and came_from == "S":
            crossings += 1
    return (crossings % 2) == 1
        
    

maps = clean_maps(maps, path)
#print_map(maps)

#maps = mark_right_left_fields(maps, path, second_pos, second_come_from)
#print_map(maps)

cells_inside = 0
for r in range(len(maps)):
    for c in range(len(maps[0])):
        if maps[r][c] == ".":
            if is_inside(maps, path, (r, c)):
                cells_inside += 1

print("Cells inside Pipe:", cells_inside)

Found Starting Point
Max Distance: 6860.0
Cells inside Pipe: 343


In [26]:
is_inside(maps, path, (5,13))

True

In [ ]:
(1,2) in path